In [8]:
# pip install spacy
# python -m spacy download en_core_web_sm
# note: i had to restart my computer after downloading the language package for it to work

In [36]:
import numpy as np
import pandas as pd
import spacy
import pyspark
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession

In [41]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

In [3]:
file = pd.read_csv("joined_data_v1.csv")

In [28]:
file = pd.DataFrame(file)
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."
6,283366,tt1935859,Miss Peregrine's Home for Peculiar Children,9/28/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Fantasy,Adventure,NaN,NaN,...,Tim Burton,https://m.media-amazon.com/images/M/MV5BMTU0Nz...,20th Century Fox,6.5,65%,67%,65%,57%,64%,Ransom Riggs (based upon the novel written by)...
8,284564,tt3835080,31,9/15/2016,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Horror,Thriller,NaN,NaN,NaN,...,Rob Zombie,https://m.media-amazon.com/images/M/MV5BNTE0NT...,Bow and Arrow Entertainment,4.9,49%,51%,49%,35%,46%,Rob Zombie


In [29]:
file.drop_duplicates(keep='first', inplace=True)

In [30]:
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."
6,283366,tt1935859,Miss Peregrine's Home for Peculiar Children,9/28/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Fantasy,Adventure,NaN,NaN,...,Tim Burton,https://m.media-amazon.com/images/M/MV5BMTU0Nz...,20th Century Fox,6.5,65%,67%,65%,57%,64%,Ransom Riggs (based upon the novel written by)...
8,284564,tt3835080,31,9/15/2016,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Horror,Thriller,NaN,NaN,NaN,...,Rob Zombie,https://m.media-amazon.com/images/M/MV5BNTE0NT...,Bow and Arrow Entertainment,4.9,49%,51%,49%,35%,46%,Rob Zombie


In [7]:
#creating a list of the overview descriptions we want to run through spacy

overview_list = file["overview"].tolist()
overview_list

['Nat Turner, a former slave in America, leads a liberation movement in 1831 to free African-Americans in Virgina that results in a violent retaliation from whites.',
 "On 15 January 2009, the world witnessed the 'Miracle on the Hudson' when Captain 'Sully' Sullenberger glided his disabled plane onto the frigid waters of the Hudson River, saving the lives of all 155 aboard. However, even as Sully was being heralded by the public and the media for his unprecedented feat of aviation skill, an investigation was unfolding that threatened to destroy his reputation and career.",
 'A story set on the offshore drilling rig Deepwater Horizon, which exploded during April 2010 and created the worst oil spill in U.S. history.',
 'A teenager finds himself transported to an island where he must help protect a group of orphans with special powers from creatures intent on destroying them.',
 "ROB ZOMBIE'S  S P O O K  H A U S 31",
 'CIA employee Edward Snowden leaks thousands of classified documents to

In [8]:
overview1 = overview_list[0]
overview1

'Nat Turner, a former slave in America, leads a liberation movement in 1831 to free African-Americans in Virgina that results in a violent retaliation from whites.'

In [87]:
overview_df = file[["overview"]]
overview_df

,overview
0,"Nat Turner, a former slave in America, leads a..."
2,"On 15 January 2009, the world witnessed the 'M..."
4,A story set on the offshore drilling rig Deepw...
6,A teenager finds himself transported to an isl...
8,ROB ZOMBIE'S S P O O K H A U S 31
10,CIA employee Edward Snowden leaks thousands of...
12,Students on a camping trip discover something ...
14,Seven gun men in the old west gradually come t...
16,A corporate risk-management consultant must de...
18,"The last Gunslinger, Roland Deschain, has been..."


In [93]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [75]:
from pyspark.sql.types import *

mySchema = StructType([StructField("overview", StringType(), True)])

In [128]:
df = spark.createDataFrame(overview_df,schema=mySchema)

In [129]:
sentenceData = spark.createDataFrame(overview_df)
sentenceData.show() 

+--------------------+
|            overview|
+--------------------+
|Nat Turner, a for...|
|On 15 January 200...|
|A story set on th...|
|A teenager finds ...|
|ROB ZOMBIE'S  S P...|
|CIA employee Edwa...|
|Students on a cam...|
|Seven gun men in ...|
|A corporate risk-...|
|The last Gunsling...|
|A stuffy business...|
|A group of teens ...|
|From DC Comics co...|
|Kubo mesmerizes t...|
|A falsely accused...|
|Based on the true...|
|A divorced dad an...|
|In December 1941,...|
|Pete is a mysteri...|
|When a radio fall...|
+--------------------+
only showing top 20 rows



In [130]:
tokenizer = Tokenizer(inputCol="overview", outputCol="words")
tokenizer

Tokenizer_fb0bf4ff7150

In [131]:
# Transform and show DataFrame
tokenized = tokenizer.transform(sentenceData)
tokenized.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|overview                                                                                                       

In [132]:
# Instantiate Remover
remover = StopWordsRemover(inputCol="words", outputCol="stopwords")

In [134]:
# Transform and show data
tokenized = remover.transform(tokenized)
tokenized.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------

# Spacy NLP

In [144]:
# Load English tokenizer, tagger, parser, NER and word vectors

nlp = spacy.load("en_core_web_sm")

In [148]:
text =str(tokenized.select("stopwords").collect()[0])
text


"Row(stopwords=['nat', 'turner,', 'former', 'slave', 'america,', 'leads', 'liberation', 'movement', '1831', 'free', 'african-americans', 'virgina', 'results', 'violent', 'retaliation', 'whites.'])"

In [149]:
doc = nlp(text)

In [162]:
# Analyze syntax
    
nouns = [chunk.text for chunk in doc.noun_chunks]
verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"]
adj = [token.lemma_ for token in doc if token.pos_ == "ADJECTIVE"]

In [163]:
print(nouns)
print(verbs)
print(adj)


["'former', 'slave', 'america", "'virgina", ", 'results', 'violent', 'retaliation', 'whites"]
["row(stopwords=['nat", 'lead']
[]


In [151]:
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

1831' DATE


In [157]:
for token in doc:
    print(token.lemma_, token.pos_)

row(stopwords=['nat VERB
' PUNCT
, PUNCT
' PUNCT
turner NOUN
, PUNCT
' PUNCT
, PUNCT
' PUNCT
former ADJ
' PUNCT
, PUNCT
' PUNCT
slave NOUN
' PUNCT
, PUNCT
' PUNCT
america NOUN
, PUNCT
' PUNCT
, PUNCT
' PUNCT
lead VERB
' PUNCT
, PUNCT
' PUNCT
liberation NOUN
' PUNCT
, PUNCT
' PUNCT
movement NOUN
' PUNCT
, PUNCT
' PUNCT
1831 NUM
' PUNCT
, PUNCT
' PUNCT
free ADJ
' PUNCT
, PUNCT
' PUNCT
african ADJ
- PUNCT
american NOUN
' PART
, PUNCT
' PUNCT
virgina NOUN
' PUNCT
, PUNCT
' PUNCT
result NOUN
' PUNCT
, PUNCT
' PUNCT
violent ADJ
' PUNCT
, PUNCT
' PUNCT
retaliation NOUN
' PUNCT
, PUNCT
' PUNCT
white NOUN
. PUNCT
' PUNCT
] PUNCT
) PUNCT
